In [1]:
# Uploaded to mapping_files folder under lr-big-lots accounts
import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
returned_df_1=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/Previous_Returned/CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz",compression="gzip",sep="|",dtype=str)
returned_df_2=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/Previous_Returned/CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz",compression="gzip",sep="|",dtype=str)

returned_df=returned_df_1.append(returned_df_2)
del returned_df_1
del returned_df_2

segment_previous_returned=returned_df['segment'].unique().tolist()

gc.collect()

7

In [4]:
my_Dec_new_signed_up_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/"
my_Dec_new_signed_up_list=[x for x in list(recursive_file_gen(my_Dec_new_signed_up_folder)) if ".csv" in x]
my_Dec_new_signed_up_list

['/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_P.csv',
 '/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_S.csv',
 '/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_T.csv']

In [5]:
SP_all_after_0922_scoring="/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/New_Segments/"
SP_new_score_list=list(recursive_file_gen(SP_all_after_0922_scoring))
set([x[len(x)-4:] for x in my_Dec_new_signed_up_list])

{'.csv'}

In [6]:
len(my_Dec_new_signed_up_list)

3

In [7]:
len(set([os.path.basename(x) for x in SP_new_score_list]))

92

In [8]:
all_file_to_be_loaded=SP_new_score_list+my_Dec_new_signed_up_list # 1229 appended
print(len(all_file_to_be_loaded))
print(len(set(all_file_to_be_loaded)))
print(len(set([os.path.basename(x) for x in all_file_to_be_loaded])))


df_all_new=pd.DataFrame()
all_file_to_be_loaded=sorted(all_file_to_be_loaded)
i_counter=0

for file_path in all_file_to_be_loaded:
    if os.path.basename(file_path)=="BL_lapsed19_48.csv":
        pass
    
    elif "Holiday_Shopper_" in os.path.basename(file_path): # To be appended below
        print(os.path.basename(file_path))
        pass
        
    else:
        df=pd.read_csv(file_path,dtype=str,usecols=['customer_id_hashed', 'email_address_hash', 'segment'])
        df=df[~df['segment'].isin(segment_previous_returned)]
        df=df.drop_duplicates('email_address_hash').drop_duplicates("customer_id_hashed")
        df_all_new=df_all_new.append(df)
    i_counter+=1
    if i_counter%10==2:
        print(i_counter)
    

95
95
95
2
12
Holiday_Shopper_Email_Non_Openers.csv
Holiday_Shopper_Email_Openers.csv
Holiday_Shopper_Full_File.csv
Holiday_Shopper_Purchase_Active.csv
Holiday_Shopper_Purchase_Inactive.csv
Holiday_Shopper_Toy_Purchase.csv
22
32
42
52
62
72
82
92


In [9]:
client_holiday='/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/New_Segments/upload_holidayshoppers/'
client_holiday_list=glob.glob(client_holiday+"*.csv")
client_holiday_df=pd.DataFrame()

for file_holiday in client_holiday_list:
    df=pd.read_csv(file_holiday,dtype=str,usecols=['customer_id_hashed', 'email_address_hash'])
    client_holiday_df=client_holiday_df.append(df)
    
client_holiday_df_dedup=client_holiday_df.drop_duplicates('email_address_hash')
client_holiday_df_dedup['segment']="Client_Holiday_Dedup"

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
df_all_new=df_all_new.append(client_holiday_df_dedup)



In [11]:
print(df_all_new.shape)
print(len(df_all_new['email_address_hash'].unique()))

(25051779, 3)
23024095


In [12]:
lapsed_raw=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip"
                         ,nrows=None,sep="|",dtype=str,usecols=['customer_id_hashed','email_address_hash'])
lapsed_raw['segment']="Lapsed_20140826_20170226"
print(len(lapsed_raw.index))
lapsed_raw=lapsed_raw.drop_duplicates(['email_address_hash'])
print(len(lapsed_raw.index))

df_all_new=df_all_new.append(lapsed_raw)

print(df_all_new.shape)
print(len(df_all_new['email_address_hash'].unique()))

1983373
1873387
(26925166, 3)
23875578


In [13]:
df_all_new=df_all_new[~df_all_new['segment'].isin(segment_previous_returned)]
df_all_new=df_all_new.drop_duplicates()
print(df_all_new.shape)
print(len(df_all_new['email_address_hash'].unique()))

(26925166, 3)
23875578


In [14]:
df_all_new_mapping=df_all_new.drop_duplicates(['email_address_hash'])



In [16]:
df_all_new_mapping.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/BL_mapping_file_2018Q4_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [17]:
df_all_new_mapping.tail(5)

,customer_id_hashed,email_address_hash,segment
1983361,6129e62df2b13e21dafdfc6c22ef6d8d88dbdcc524d7f7...,f836e783f10e7203f0787ab3047910dc2c4be9d69665c8...,Lapsed_20140826_20170226
1983362,ab1684570a82a749823b38606cf76f5ffecabbf4e91156...,5c439077025f54555b7270e5ed8814150b69b88ecd785e...,Lapsed_20140826_20170226
1983363,7b0301d5306e13b274ce2281d288f65be064d6d4eb48b9...,361eb6f1c4c6be4edb1ee21dbf85a9c22c9a79f0a80445...,Lapsed_20140826_20170226
1983364,61c5050a2c3f480106d158849cdaf97b361ca66fba069f...,7065c3a87e08258926ad1be3ca8bf7f59c326ec895004e...,Lapsed_20140826_20170226
1983371,7e79e30f59d1a1f387c618b1decd7d052fe33aac6d9549...,0afb79d21359783ebd4012a20cf481067d9bb9b2307ad8...,Lapsed_20140826_20170226
